In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.optimize import curve_fit
from scipy.integrate import cumulative_trapezoid
from scipy.interpolate import CubicSpline

In [ ]:
class IsingModel:
    def __init__(self, L, T):
        #parametros do modelo
        self.L = L
        self.T = T

        #array com os spins
        self.spins = np.ones((L,L))

        #arrays para guardar evoluções das variáveis
        self.energies= []
        self.magnetizations= [-L**2]

        #energia e magnetização total
        self.energy_total = self.calc_ener_total()
        self.magnet_total = np.sum(self.spins)

    def calc_ener_spin(self, i, j):
        #calcular a energia de um spin
        spin= self.spins[i,j]
        neighbours= (
        self.spins[(i+1)%self.L, j] + 
        self.spins[i, (j+1)%self.L] + 
        self.spins[(i-1)%self.L, j] + 
        self.spins[i, (j-1)%self.L]
        )
        energy=-spin*neighbours
        return energy
    
    def calc_ener_total( self ):
        energy=0
        for i in range(self.L):
            for j in range(self.L):
                energy+= -self.spins[i,j]*(self.spins[(i+1)%self.L,j]+ self.spins[i,(j+1)%self.L])
        return energy
    
    def calc_ener ( self ):
        energy=self.energy_total/self.L**2

        return energy
    
    def calc_mag( self ):
        mag = self.magnet_total
        return mag
    
    def iter_monte_carlo ( self, n_iter ):

        for i in tqdm (range(n_iter), desc=f"L={self.L:6d}, T={self.T:8f}"):
    
    @property
    def energy(self):
        return np.array(self.energies)
    


## 1. Tempo de Termalização

**a)** O código corre uma simulação para cada combinação de L e T e dá plot no gráfico da energia média.

In [ ]:
L_array = [16, 32, 64, 128]
T_array = [1, 2, 3, 4]
ener_array = np.zeros(4)

for L in L_array:
    for T in T_array:
        ising = IsingModel(L, T)
        n_iter = np.linspace(1, 10, 9)
        sims = ising.iter_monte_carlo(n_iter) #Correr a simulação para cada n_iter
        avg_ener = (ising.energies/L) #Energia média
        np.append(ener_array, avg_ener)

        plt.figure(figsize=(10, 5))
        plt.plot(n_iter, avg_ener, label=f'Simulação {L}-{T}', color='blue') 
        plt.title('Gráfico de Seno') 
        plt.xlabel('Eixo X') 
        plt.ylabel('Eixo Y') 
        plt.legend() 
        plt.grid() 
        plt.show()

**b)** Fit

In [ ]:
def ener_func(N, e0, ef, tau):
    return ef + (e0 - ef) * np.exp(- N/tau)

vars = [curve_fit(ener_func, n_iter, ener_array[i]) for i in range(4)]

In [ ]:
taus = vars[2]

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(L, T, taus, cmap='viridis', edgecolor='none')
ax.set_xlabel('L')
ax.set_ylabel('T')
ax.set_zlabel(r'$\tau_{term}$')
ax.set_title('Intensidade da Luz em Função de X e Y')
plt.show() # mostrando o gráfico


## 2. Determinação da Temperatura Crítica

## 3. Determinação da Energia Livre de Helmholtz, F